In [ ]:
# load transcription data and save 'test' to test_filelist.txt
import pandas as pd
import os
import sys
sys.path.append('../')

# load transcription data
data_df = pd.read_csv('../datasets/MC-EIU_Track1_English_Processed/transcription.csv')
test_data_df = data_df[data_df['name'].str.contains('Testing')]
test_data_df = test_data_df[['name']]
test_data_df.to_csv('../filelists/Track1_English_origin/test_filelist.txt', sep=' ', header=False, index=False)

In [ ]:
# load dataset
from data.multimodal_dataset import MultimodalDataset
import argparse

opt = argparse.Namespace(
    name='my_baseline',
    log_dir='logs',
    checkpoint_dir='checkpoints',
    gpu_id='0',
    model='baseline',
    input_feature_dims=[512, 1024, 768],
    emotion_network_names=['LSTMEncoder', 'TextCNN', 'LSTMEncoder'],
    emotion_network_embed_dims=[128, 128, 128],
    intent_network_names=['LSTMEncoder', 'TextCNN', 'LSTMEncoder'],
    intent_network_embed_dims=[128, 128, 128],
    hidden_size=128,
    dataset_filelist_dir='../filelists/Track1_English_origin',
    feature_dirs=[
        '../datasets/MC-EIU_Track1_English_Processed/Features/wav2vec-large-c-FRA',
        '../datasets/MC-EIU_Track1_English_Processed/Features/manet_FRA',
        '../datasets/MC-EIU_Track1_English_Processed/Features/roberta-base-4-FRA'
    ],
    niter=20,
    niter_decay=40,
    print_freq=10,
    batch_size=32,
    pretrained_path='../checkpoints/my_baseline_2024-10-25_23-23-34_baseline/best.pth',
    ablation='normal',
)

dataset = MultimodalDataset(opt, set_name='test')

In [ ]:
# load model and predict
from models.models import BaselineModel
import torch

emotions = ['happy', 'surprise', 'sad', 'disgust', 'anger', 'fear', 'neutral']
intents = ['questioning', 'agreeing', 'acknowledging', 'encouraging', 'consoling', 'suggesting', 'wishing', 'neutral']

emo2idx, idx2emo = {}, {}
int2idx, idx2int = {}, {}

for ii, emo_label in enumerate(emotions): emo2idx[emo_label] = ii
for ii, emo_label in enumerate(emotions): idx2emo[ii] = emo_label

for ii, int_label in enumerate(intents): int2idx[int_label] = ii
for ii, int_label in enumerate(intents): idx2int[ii] = int_label

submission_datas = []

model = BaselineModel(opt).to('cuda:0')
for i, data in enumerate(dataset):
    with torch.no_grad():
        features = data['features']
        features = [f.unsqueeze(0).to('cuda:0') for f in features]
        pred = model.predict(features)
        pred_emotion_label_id = pred[0].argmax(dim=1)
        pred_intent_label_id = pred[1].argmax(dim=1)
        submission_data = [data['filename'], idx2emo[pred_emotion_label_id.item()], idx2int[pred_intent_label_id.item()]]
        submission_datas.append(submission_data)

In [ ]:
submission_df = pd.DataFrame(submission_datas, columns=['filename', 'emo_pred', 'int_pred'])
submission_csv_dir = f'../{opt.log_dir}/{opt.pretrained_path.split("/")[2]}'
submission_df.to_csv(f'{submission_csv_dir}/submission.csv', index=False)
# zip submission.csv to submission.zip in a defined directory
os.system(f'zip submission.zip {submission_csv_dir}/submission.csv')